In [70]:
! pip install scikit-surprise
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import surprise
from surprise import Reader, Dataset, SVD, evaluate
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Charlotte\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\CHARLO~1\\AppData\\Local\\Temp\\pip-install-5elw9rr1\\scikit-surprise_58b07b9ad4904f7087df2bf97fd9b46e\\setup.py'"'"'; __file__='"'"'C:\\Users\\CHARLO~1\\AppData\\Local\\Temp\\pip-install-5elw9rr1\\scikit-surprise_58b07b9ad4904f7087df2bf97fd9b46e\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\CHARLO~1\AppData\Local\Temp\pip-wheel-mndx2c56'
       cwd: C:\Users\CHARLO~1\AppData\Local\Temp\pip-install-5elw9rr1\scikit-surprise_58b07b9ad4904f7087df2bf97fd9b46e\
  Complete output (52 lines):
  C:\Users\Charlotte\Anaconda3\lib\site-packages\setuptools\dist.py:720: U

ModuleNotFoundError: No module named 'surprise'

In [50]:
df = pd.read_csv("C:/Users/Charlotte/Desktop/TSE/M2/MATHS OF DEEP LEARNING/projet_recom/steam-200k.csv",
    header=None,
    names=["userid", "game", "action", "purchase_or_hours", "useless"]
)[["userid", "game", "action", "purchase_or_hours"]]
df.head()

,userid,game,action,purchase_or_hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


Note: purchase_or_hours always takes the value "1" when the game was purchased and the number of hours played otherwise.

Missing Values

In [51]:
df.isnull().sum()

userid               0
game                 0
action               0
purchase_or_hours    0
dtype: int64

Appreciation
In this dataset, we do not have an explicit feature translating the rate of appreciation of a game by a user. Therefore, we consider that the user liked the game if he purchased it or played at least 20 hours.
God knows there are video games we hated but still played for more than 20 hours, which justifies a higher threshold. However, that would unvoluntarily discard small video games (which take few hours to complete).

Quoique je me dis ptet que c'est mieux :

3 if plays more than 50 hours (even if it's a small game, if it's really good then it can be played multiple times) purchased or not
2 if between 20 and 50 hours, purchased or not (financial reasons not allowing purchase)
1 if purchase AND less than 20 hours (likes it but not so much)
0 if no purchase AND less than 20 hours (does not like it)
on peut réduire le nombre de classes si problématique

In [52]:
df["score"] = [
    3 if df["purchase_or_hours"][i] >= 50 else
    2 if df["purchase_or_hours"][i] >= 20 and df["purchase_or_hours"][i] < 50 else
    1 if df["purchase_or_hours"][i] < 20 and df["action"][i] == "purchase" else
    0 for i in range(len(df))
]

In [53]:
df

,userid,game,action,purchase_or_hours,score
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,1
1,151603712,The Elder Scrolls V Skyrim,play,273.0,3
2,151603712,Fallout 4,purchase,1.0,1
3,151603712,Fallout 4,play,87.0,3
4,151603712,Spore,purchase,1.0,1
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,1
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,1


Toy-model with TMDB Ratings (directly from https://www.kaggle.com/rounakbanik/movie-recommender-systems)

Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

    - $v$ the number of scores for a game
    - $m$ the minimum number of scores required to be listed in the chart
    - $R$ the average score of a game
    - $C$ the average score across the whole games library
    
    We try a first version of the TMDB Ratings formula with $m$ defined as the 95th percentile of the score variable 
   

In [54]:
score_counts = df[df['score'].notnull()]['score'].astype('int')
score_averages = df.groupby(by = "game")["score"].mean()
C = score_averages.mean()
C

0.8224353708954674

In [55]:
score_averages
df_sa = pd.DataFrame(score_averages)
df_sa
df_sa = df_sa.reset_index()
df_sa.rename(columns = {'score':'score_averages',inplace = True)

SyntaxError: invalid syntax (<ipython-input-55-b849032e471e>, line 5)

In [ ]:
df_sa.rename(columns = {'score':'score_averages'},inplace = True)
df_sa

In [ ]:
df_tot = df.merge(df_sa, 'inner', 'game')
df_tot

In [ ]:
# plot top 10 scores
# follow kaggle eg

In [ ]:
df_tot.dtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np

sns.set(rc={"figure.figsize": (8, 4)}); np.random.seed(0)
ax = sns.distplot(df_tot['score_averages'])
plt.show()

In [57]:
m = score_counts.quantile(0.95)
m

2.0

In [56]:
qualified = df_tot[(df_tot['score'] >= m) & (df_tot['score'].notnull()) & (df_tot['score_averages'].notnull())][['userid', 'game', 'action', 'purchase_or_hours', 'score', 'score_averages']]
qualified['score'] = qualified['score'].astype('int')
qualified['score_averages'] = qualified['score_averages'].astype('int')
qualified.shape

(17183, 6)

In [58]:
qualified['score_averages'].mean()

0.46278298318105104

In [ ]:
qualified

In [ ]:
df_tot

In [46]:
df_tot.shape

(200000, 6)

17183 video games are considered on our chart, with an average score of 0.46. 

In [59]:
def weighted_rating(x):
    v = x['score']
    R = x['score_averages']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [60]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified

,userid,game,action,purchase_or_hours,score,score_averages,wr
1,151603712,The Elder Scrolls V Skyrim,play,273.0,3,1,0.928974
3,59945701,The Elder Scrolls V Skyrim,play,58.0,3,1,0.928974
6,92107940,The Elder Scrolls V Skyrim,play,110.0,3,1,0.928974
8,250006052,The Elder Scrolls V Skyrim,play,465.0,3,1,0.928974
10,11373749,The Elder Scrolls V Skyrim,play,220.0,3,1,0.928974
...,...,...,...,...,...,...,...
199742,246319849,NOBUNAGA'S AMBITION Kakushin with Power Up Kit,play,267.0,3,2,1.528974
199744,74772820,Eschalon Book 1,play,57.0,3,2,1.528974
199802,42380987,Wildlife Park 2,play,29.0,2,1,0.911218
199920,213854339,Eastside Hockey Manager,play,1295.0,3,2,1.528974


In [61]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [62]:
qualified

,userid,game,action,purchase_or_hours,score,score_averages,wr
199964,42657809,Worldwide Soccer Manager 2009,play,295.0,3,2,1.528974
196299,46055854,FIFA Manager 11,play,229.0,3,2,1.528974
198631,165240612,Cultures - Northland,play,194.0,3,2,1.528974
198314,88199811,RPG Maker MV,play,55.0,3,2,1.528974
198004,71527252,Runers,play,50.0,3,2,1.528974
...,...,...,...,...,...,...,...
64935,66596291,Call of Duty Modern Warfare 2 - Multiplayer,play,536.0,3,1,0.928974
64931,21861124,Call of Duty Modern Warfare 2 - Multiplayer,play,968.0,3,1,0.928974
64924,77542968,Call of Duty Modern Warfare 2 - Multiplayer,play,128.0,3,1,0.928974
64920,31669242,Call of Duty Modern Warfare 2 - Multiplayer,play,249.0,3,1,0.928974


Without rescaling the score index using the Weighted Rating transformation, we would not have been able to clearly discriminate between games with purchase_or_hours score greater than 50 e.g.Worldwide Soccer Manager 2009 and Call of Duty Modern Warfare 2 - Multiplayer.

But by taking into account the average score of each game, that is their overall rating by the userbase, we observe that Worldwide Soccer Manager 2009, FIFA Manager 11, 	Cultures - Northland, 	RPG Maker MV and Runers
are associated with the highest Weighted Rating scores, with only 823 cumulated hours played. 

On the contrary, the Call of Duty saga yields the smallest Weighted Rating scores, 0.92 on average, with almost 2400 hours played in total.

## Collaborative Filtering

In [73]:
algo = SVD()

# Run 5-fold cross-validation and print results.

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(algo, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(df_tot)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

NameError: name 'SVD' is not defined

In [ ]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(df_tot.build_full_trainset())

In [ ]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [ ]:
results_df

KNN Algorithm

In [ ]:
# try https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline

# try hybrid recommender: DO NOT RUN

In [ ]:
def hybrid(userid, game):
    idx = df_tot['userid']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
df_tot = df_tot.set_index('userid')

In [ ]:
# to be modified and adapated to our dataset

def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
# try
# https://surprise.readthedocs.io/en/stable/matrix_factorization.html
# https://surprise.readthedocs.io/en/stable/similarities.html